In [10]:
import cv2
from typing import *


def Downscale(img, target_dim: Tuple[int, int], kernel_size: Tuple[int, int], scale_factor: float = 2.0):
    """
    Downscaling source/target y ∈ Y domain --> y↓ (LR img, same dimention as x)
    :param img:
    :param target_dim: dimension after bicubic downscaling
    :param scale_factor: constant that we choose at the beginning of the algorithm
    :param kernel_size: size of blurring kernel from Gauss distribution
    :return:
    """
    sigma = scale_factor / 2.0      # like in the article
    blurred = cv2.GaussianBlur(img, kernel_size, sigmaX=sigma, sigmaY=sigma)
    resized = cv2.resize(blurred, target_dim, interpolation=cv2.INTER_AREA)

    return resized

In [94]:
"""
Discriminator and Generator implementation from DCGAN paper
"""

import torch
import torch.nn as nn


class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # input: N x channels_img x 32 x 32
            nn.Conv2d(
                channels_img, features_d, kernel_size=4, stride=2, padding=1
            ),
            nn.LeakyReLU(0.2),
            # _block(in_channels, out_channels, kernel_size, stride, padding)
            self._block(features_d, features_d * 2, 4, 2, 1),
            self._block(features_d * 2, features_d * 4, 4, 2, 1),
#             self._block(features_d * 4, features_d * 8, 4, 2, 1),
            # After all _block img output is 4x4 (Conv2d below makes into 1x1)
            nn.Conv2d(features_d * 4, 1, kernel_size=4, stride=2, padding=0),
            nn.Sigmoid(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.disc(x)


class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            # Input: N x channels_noise x 1 x 1
            self._block(channels_noise, features_g * 16, 4, 1, 0),  # img: 4x4
            self._block(features_g * 16, features_g * 8, 4, 2, 1),  # img: 8x8
            self._block(features_g * 8, features_g * 4, 4, 2, 1),  # img: 16x16
#             self._block(features_g * 4, features_g * 2, 4, 2, 1),  # img: 32x32
            nn.ConvTranspose2d(
                features_g * 4, channels_img, kernel_size=4, stride=2, padding=1
            ),
            # Output: N x channels_img x 32 x 32
            nn.Tanh(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)


def initialize_weights(model):
    # Initializes weights according to the DCGAN paper
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)

def test():
    N, in_channels, H, W = 8, 3, 32, 32
    noise_dim = 100
    x = torch.randn((N, in_channels, H, W))
    disc = Discriminator(in_channels, 8)
    assert disc(x).shape == (N, 1, 1, 1), "Discriminator test failed"
    gen = Generator(noise_dim, in_channels, 8)
    z = torch.randn((N, noise_dim, 1, 1))
    assert gen(z).shape == (N, in_channels, H, W), "Generator test failed"


test()

In [87]:
from sklearn.utils import shuffle
from PIL import Image
import numpy as np

RANDOM_SEED = 228


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform):
        self.data_dir = data_dir        # example: 'dataset/train'
        self.transform = transform    
        # вот тут я просто сгенерю вектора со всеми картинками (офк путями к ним)
        # и буду просто по ним итерироваться
        self.labeled_objects = []   # в формате (<путь до файла>, <номер класса>)
        for dirname, dirs, files in os.walk(self.data_dir):
            for filename in files:
                fname_without_ext, extension = os.path.splitext(filename)
                self.labeled_objects.append((join(dirname, filename),
                                             int(fname_without_ext)))   

        self.labeled_objects = shuffle(self.labeled_objects, 
                                       random_state=RANDOM_SEED)
        
    def __getitem__(self, idx):
        img_name = self.labeled_objects[idx][0]
        
#         img = cv2.imread(img_name)
#         image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        image = Image.open(img_name)
        label = torch.tensor([self.labeled_objects[idx][1]])

        # отрисовка картиноки до аугментации
#         fig=plt.figure(figsize=(8, 8))
#         fig.add_subplot(1, 3, 1)
#         plt.imshow(np.asarray(image))
#         image = Downscale(image, (64, 64), (101, 101))
        
        
        if self.transform is not None:
            image = self.transform(image)

        # отрисовка картиноки после аугментации
#         fig.add_subplot(1, 3, 2)
#         plt.imshow(image)    
        
        return image, label
    def __len__(self):
        return len(self.labeled_objects)

In [71]:
import os
from os.path import join

root="../../DATA/archive/"

In [82]:
import matplotlib.pyplot as plt

train_dataset = MyDataset(root + 'DIV2K_train_HR', transforms)
val_dataset = MyDataset(root + 'DIV2K_valid_HR', transforms)

obg_num = 11


img, label = train_dataset[obg_num]
img, label = val_dataset[obg_num]

torch.Size([3, 64, 64])
torch.Size([3, 64, 64])


In [83]:
# Just very simple checks
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
print("tests passed")

torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
tests passed


In [96]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from torch.utils.tensorboard import SummaryWriter

# from model import Discriminator, Generator, initialize_weights

# Hyperparameters etc.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 2e-4  # could also use two lrs, one for gen and one for disc
BATCH_SIZE = 64
IMAGE_SIZE = 32
CHANNELS_IMG = 3
NOISE_DIM = 100

NUM_EPOCHS = 20

FEATURES_DISC = 64
FEATURES_GEN = 64

transforms = transforms.Compose(
    [
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE), interpolation=4),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]
        ),
        
    ]
)

# If you train on MNIST, remember to set channels_img to 1
# dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms,
#                        download=True)

# train_dataset = datasets.ImageFolder(root="../../DATA/archive/DIV2K_train_HR",transform=train_transform)

dataset = MyDataset(root + 'DIV2K_train_HR', transforms)

# comment mnist above and uncomment below if train on CelebA
# dataset = datasets.ImageFolder(root="celeb_dataset", transform=transforms)

# dataset = datasets.ImageFolder(root=root + 'DIV2K_train_HR')

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)


gen = Generator(NOISE_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)
initialize_weights(gen)
initialize_weights(disc)

opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
criterion = nn.BCELoss()

fixed_noise = torch.randn(32, NOISE_DIM, 1, 1).to(device)
writer_real = SummaryWriter(f"logs_b64/real")
writer_fake = SummaryWriter(f"logs_b64/fake")
step = 0

gen.train()
disc.train()

for epoch in range(NUM_EPOCHS):
    # Target labels not needed! <3 unsupervised
    # enumerate
    # batch_idx
    for batch_idx, (real, _) in enumerate(dataloader):
        real = real.to(device)
        noise = torch.randn(BATCH_SIZE, NOISE_DIM, 1, 1).to(device)
        fake = gen(noise)

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
#         print(real.shape)
#         print(disc(real).shape)
        disc_real = disc(real).reshape(-1)
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake.detach()).reshape(-1)
        loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        loss_disc = (loss_disc_real + loss_disc_fake) / 2
        disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()

        ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        output = disc(fake).reshape(-1)
        loss_gen = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

        # Print losses occasionally and print to tensorboard
        if batch_idx % 100 == 0:
            print(
                f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} \
                  Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise)
                # take out (up to) 32 examples
                img_grid_real = torchvision.utils.make_grid(
                    real[:32], normalize=True
                )
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:32], normalize=True
                )

                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step=step)

            step += 1

Epoch [0/20] Batch 0/13                   Loss D: 0.6926, loss G: 0.7325
Epoch [1/20] Batch 0/13                   Loss D: 0.5322, loss G: 0.9881
Epoch [2/20] Batch 0/13                   Loss D: 0.3795, loss G: 1.2744
Epoch [3/20] Batch 0/13                   Loss D: 0.2818, loss G: 1.5440
Epoch [4/20] Batch 0/13                   Loss D: 0.2040, loss G: 1.8304
Epoch [5/20] Batch 0/13                   Loss D: 0.1834, loss G: 1.9489
Epoch [6/20] Batch 0/13                   Loss D: 0.1844, loss G: 2.2740
Epoch [7/20] Batch 0/13                   Loss D: 0.1157, loss G: 2.5419
Epoch [8/20] Batch 0/13                   Loss D: 0.3272, loss G: 2.1414
Epoch [9/20] Batch 0/13                   Loss D: 0.3229, loss G: 2.0600
Epoch [10/20] Batch 0/13                   Loss D: 0.3820, loss G: 2.4735
Epoch [11/20] Batch 0/13                   Loss D: 0.3533, loss G: 1.9665
Epoch [12/20] Batch 0/13                   Loss D: 0.3033, loss G: 2.6421
Epoch [13/20] Batch 0/13                   Loss 

In [98]:
noise.shape

torch.Size([64, 100, 1, 1])

In [93]:
ls

dataset/  logs/  train_loop.ipynb  Untitled.ipynb


100 3 64
Epoch [0/20] Batch 0/7                   Loss D: 0.6916, loss G: 0.7954
Epoch [1/20] Batch 0/7                   Loss D: 0.4763, loss G: 1.1888
Epoch [2/20] Batch 0/7                   Loss D: 0.3154, loss G: 1.5132
Epoch [3/20] Batch 0/7                   Loss D: 0.2026, loss G: 1.8465
Epoch [4/20] Batch 0/7                   Loss D: 0.1434, loss G: 2.1313
Epoch [5/20] Batch 0/7                   Loss D: 0.1057, loss G: 2.4202
Epoch [6/20] Batch 0/7                   Loss D: 0.0830, loss G: 2.6598
Epoch [7/20] Batch 0/7                   Loss D: 0.0613, loss G: 2.8991
Epoch [8/20] Batch 0/7                   Loss D: 0.0697, loss G: 3.1156
Epoch [9/20] Batch 0/7                   Loss D: 0.0413, loss G: 3.2914
Epoch [10/20] Batch 0/7                   Loss D: 0.0350, loss G: 3.4567
Epoch [11/20] Batch 0/7                   Loss D: 0.0329, loss G: 3.5475
Epoch [12/20] Batch 0/7                   Loss D: 0.1851, loss G: 2.6173
Epoch [13/20] Batch 0/7                   Loss D: 0.0830, loss G: 3.0530
Epoch [14/20] Batch 0/7                   Loss D: 0.0504, loss G: 3.3660
Epoch [15/20] Batch 0/7                   Loss D: 0.0506, loss G: 3.3980
Epoch [16/20] Batch 0/7                   Loss D: 0.0374, loss G: 3.6224
Epoch [17/20] Batch 0/7                   Loss D: 0.0279, loss G: 3.7758
Epoch [18/20] Batch 0/7                   Loss D: 0.0233, loss G: 3.8754
Epoch [19/20] Batch 0/7                   Loss D: 0.0254, loss G: 3.9450
